In [1]:
import os
import sys
sys.path.append(os.path.curdir)
os.chdir('..')
import core.ssh
import getpass

In [2]:
Pkeypassword = getpass.getpass("password:")
userName = 'ggr_yz' 
#key1 = getpass.getpass("key1:")
#key2 = getpass.getpass("key2:")
key1 = '4G8F4PBHBLNX7ZOW8N5P'
key2 = 'N7u4Xwl6UUf6cV5y2G0KxWse6MC24VHjNHm8f0Lr'
bucket = "ggr-bucket-cbf77f1e-eea2-4b4a-88b2-ae787daf3f42" 
rsa_id_path = "C:\\Users\\zieft\\Dropbox\\MA\\SSHkey\\id_rsa"
folder = 'ggr_yz_ArUco2_large'
mgFileName = 'aluTest3.mg'

password:········


In [3]:
print(core.ssh.printTime(), 'SSH connection established!')
sshSession, workdir = core.ssh.PKeyLogin_Trans(rsa_id_path, '{}'.format(Pkeypassword), '137.226.78.226', 22, userName)

[13-09-2021 16:37:41] SSH connection established!


In [4]:
print(core.ssh.printTime(), 'Downloading dataset to the master node...')
core.ssh.k8_3s3download(sshSession, key1, key2, folder) 

[13-09-2021 16:37:43] Downloading dataset to the master node...
../mc
ggr_yz_ArUco2_large/
ggr_yz_ArUco2_large/2021-09-07_14-30-01-1631021401_H0.0_V0.0_C1.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-03-1631021403_H0.0_V0.0_C2.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-05-1631021405_H0.0_V5.0_C1.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-07-1631021407_H0.0_V5.0_C2.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-10-1631021410_H0.0_V10.0_C1.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-12-1631021412_H0.0_V10.0_C2.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-14-1631021414_H0.0_V15.0_C1.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-15-1631021415_H0.0_V15.0_C2.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-18-1631021418_H0.0_V20.0_C1.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-20-1631021420_H0.0_V20.0_C2.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-23-1631021423_H0.0_V25.0_C1.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-25-1631021425_H0.0_V25.0_C2.tiff
ggr_yz_ArUco2_large/2021-09-07_14-30-27-1631021427_H0.0_V30.0_C1.tiff
ggr

In [5]:
print(core.ssh.printTime(), 'Creating Persistent Volume and meshroom container...')
core.ssh.kubectlApply(sshSession, '{}/wzlk8toolkitCache/k8_3configuration/pvcJobMeshroom.yaml'.format(workdir))

[13-09-2021 16:39:37] Creating Persistent Volume and meshroom container...
persistentvolumeclaim/yz-pvc created
job.batch/yz-meshroom created

[13-09-2021 16:39:38] Success!


In [6]:
print(core.ssh.printTime(), 'Getting pod name...')
fullPodName = core.ssh.kubectlGetFullPodName(sshSession, 'yz-meshroom')
print(core.ssh.printTime(), 'Pod name is: ', fullPodName)

[13-09-2021 16:41:13] Getting pod name...
[13-09-2021 16:41:13] Pod name is:  yz-meshroom-8dszx


In [7]:
print(core.ssh.printTime(), 'Moving dataset into computation units...')
core.ssh.kubectlCp(sshSession, '{}/wzlk8toolkitCache/{}'.format(workdir, folder), fullPodName, '{}'.format(folder))

[13-09-2021 16:41:15] Moving dataset into computation units...
[13-09-2021 16:41:56] Success!


In [8]:
print(core.ssh.printTime(), 'Initiating computing environment...')
_,stdout,_ = sshSession.exec_command('kubectl exec -n ggr {} -- mv /opt/AliceVision_install /opt/AliceVision_bundle'.format(fullPodName))
print(stdout.read().decode())
core.ssh.getstatus(sshSession)

[13-09-2021 16:41:56] Initiating computing environment...
nvidia 34037760 87 nvidia_modeset,nvidia_uvm, Live 0xffffffffc357c000 (POE)

[13-09-2021 16:43:04] Success!


In [9]:
if mgFileName != '':
    """
    given .mg file should be put in the raw dataset folder and must be generated or
    converted from Meshroom version 2021.1!
    """
    # generate a project .mg file from dataset
    print(core.ssh.printTime(), 'Generating a standard .mg file.')
    _, stdout, _ = sshSession.exec_command('kubectl exec -n ggr {} -- meshroom_batch -i /tmp/{} --compute no --save /tmp/{}/generatedMgTemplate.mg -o /tmp/MeshroomCache'.format(fullPodName, folder, folder))
    print(core.ssh.printTime(), stdout.read().decode())

    # create a shell script in k8-3
    sshSession.exec_command('echo "#!/bin/sh\npython /tmp/{}/mgFileEditor.py /tmp/{}/{} {}" > {}/wzlk8toolkitCache/mgFileTemplates/runpython.sh'.format(folder, folder, mgFileName, folder, workdir, ))

    # copy the shell script into container, this script is used to run python scripts inside container.
    core.ssh.kubectlCp(sshSession, '{}/wzlk8toolkitCache/mgFileTemplates/runpython.sh'.format(workdir), fullPodName, '{}'.format(folder))

    # copy project mgFile into container
    core.ssh.kubectlCp(sshSession, '{}/wzlk8toolkitCache/mgFileTemplates/{}'.format(workdir, mgFileName), fullPodName, '{}'.format(folder))

    print(core.ssh.printTime(), 'Inserting customized computing graph')
    urlToMgFileEditer = 'https://raw.githubusercontent.com/zieft/wzlk8toolkit/master/Scripts/mgFileEditor.py'
    _, stdout, _ = sshSession.exec_command('kubectl exec -n ggr {} -- wget -P /tmp/{} {}'.format(fullPodName, folder, urlToMgFileEditer))
    print(core.ssh.printTime(), stdout.read().decode())

    _, stdout, _ = sshSession.exec_command('kubectl exec -n ggr {} -- bash /tmp/{}/runpython.sh'.format(fullPodName, folder))
    print(core.ssh.printTime(), stdout.read().decode())
    core.ssh.getstatus(sshSession)


    print(core.ssh.printTime(), 'Computing...This process may take hours to finish, please wait...')
    _, stdout, _ = sshSession.exec_command('kubectl exec -n ggr {} -- meshroom_compute --cache /tmp/MeshroomCache /tmp/{}/{}'.format(fullPodName, folder, mgFileName))
    print(core.ssh.printTime(), stdout.read().decode())

else:
    # run a standard pipeline
    print(core.ssh.printTime(), 'Computing...Please wait...')
    _,stdout,_ = sshSession.exec_command('kubectl exec -n ggr {} -- meshroom_batch -i /tmp/{} -o /tmp/MeshroomCache/{}_out'.format(fullPodName, folder, folder))
    print(stdout.read().decode())
    core.ssh.getstatus(sshSession)

[13-09-2021 16:43:04] Generating a standard .mg file.
[13-09-2021 16:43:04] nvidia 34037760 87 nvidia_modeset,nvidia_uvm, Live 0xffffffffc357c000 (POE)
Program called with the following parameters:
 * allowSingleView = 1
 * allowedCameraModels = "pinhole,radial1,radial3,brown,fisheye4,fisheye1"
 * defaultCameraModel = "" (default)
 * defaultFieldOfView = 45
 * defaultFocalLengthPix = -1 (default)
 * defaultIntrinsic = "" (default)
 * groupCameraFallback =  Unknown Type "20EGroupCameraFallback"
 * imageFolder = "" (default)
 * input = "/tmp/tmptmwjcgbw/CameraInit/f9436e97e444fa71a05aa5cf7639b206df8ba282//viewpoints.sfm"
 * output = "/tmp/tmptmwjcgbw/CameraInit/f9436e97e444fa71a05aa5cf7639b206df8ba282/cameraInit.sfm"
 * sensorDatabase = "/opt/Meshroom_bundle/aliceVision/share/aliceVision/cameraSensors.db"
 * useInternalWhiteBalance = 1
 * verboseLevel = "info"
 * viewIdMethod =  Unknown Type "N11aliceVision9sfmDataIO13EViewIdMethodE"
 * viewIdRegex = ".*?(\d+)" (default)

File successful

<img src="./521.png" width = "500" height = "400" alt="1" align=left />

In [10]:
print(core.ssh.printTime(), 'Dismounting finished Job...Please wait...')
core.ssh.kubectlDelete(sshSession, 'job', 'yz-meshroom')

[13-09-2021 16:53:54] Dismounting finished Job...Please wait...
[13-09-2021 16:53:54] Success!


In [ ]:
# create a minio server in the cluster, expose port 9000 as service
print(core.ssh.printTime(), 'Creating minIO Service... Waiting...')
core.ssh.kubectlApply(sshSession, '{}/wzlk8toolkitCache/k8_3configuration/svcForUpload.yaml'.format(workdir))

In [ ]:
# Get IP address of the host machine from minIO service
svcIP = core.ssh.getSvcIp(sshSession)

In [ ]:
# create an alias in minio client on master node.
print(core.ssh.printTime(), 'Creating myminio in Minio Client...')
core.ssh.mcAliasSet(sshSession, workdir, 'myminio', svcIP, key1, key2, 's3v4')

In [ ]:
# upload output data to s3 storage
print(core.ssh.printTime(), 'Uploading output data to S3 Storage...')
pathJob = 'MeshroomCache'
core.ssh.mcUpload(sshSession, key1, key2, pathJob, bucket, folder)

In [11]:
print(core.ssh.printTime(), 'Job finished, clearing cache...')
core.ssh.kubectlDelete(sshSession, 'job', 'minio-server')
core.ssh.kubectlDelete(sshSession, 'service', 'minio-service')
core.ssh.kubectlDelete(sshSession, 'persistentvolumeclaim', 'yz-pvc')
# remove cache folders/scripts in k8-3
sshSession.exec_command('rm -rf wzlk8toolkitCache')
core.ssh.getstatus(sshSession)

# close session
sshSession.close()
print(core.ssh.printTime(), 'All done!')

[13-09-2021 16:53:55] Job finished, clearing cache...
[13-09-2021 16:53:55] Success!
[13-09-2021 16:53:55] Success!
[13-09-2021 16:53:55] Success!
[13-09-2021 16:53:56] Success!
[13-09-2021 16:53:56] All done!
